In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import numpy as np
import pandas as pd

# デバッグ

In [4]:
kabutan_url = 'https://kabutan.jp/'
kabutan_kobetu = 'https://kabutan.jp/stock/?code='
code = '9505'

In [5]:
response = requests.get(kabutan_kobetu + code)
soup = BeautifulSoup(response.content, "html.parser")

### PBRなど

In [26]:
tag = soup.find('div', id = "stockinfo_i3")
tag_tds = tag.find_all('td')

text_list = []
for td in tag_tds:
  # print(td.get_text())
  text_list.append(td.get_text())

9.6倍
0.71倍
－％
10.29倍
1,931億円


### 乖離率

In [27]:
tag = soup.find('div', class_="kabuka_trend clearfix")
tr = tag.find_all('tr')[-1]
tds = tr.find_all('td')

text_list = []
for td in tds:
  print(td.get_text()[:-1])
#   text_list.append(td.get_text()[:-1])

+5.86
+10.37
+14.32
+37.74


### 出来高など

In [32]:
tag = soup.find('div', id="kobetsu_left")
tables = tag.find_all('table')
for table in tables[:2]:
    trs = table.find_all('tr')
    for tr in trs:
        print(tr.find('td').get_text())

885.8
925.4
885.0
918.3
2,214,000 株
2,016 百万円
910.787 円
5,257 回


91,830 円
100 株
1,931億円
210,333,694 株


### 業種

In [16]:
tag = soup.find('div', class_="company_block")
trs = tag.find_all('tr')
trs[3].td.get_text()

'銀行業'

### string修正関数

In [36]:
# import re

# # サンプルデータ
# data = [
#     '885.8',
#     '925.4',
#     '885.0',
#     '918.3',
#     '2,214,000 株',
#     '2,016 百万円',
#     '910.787 円',
#     '5,257 回'
# ]

# 数字の部分だけを抽出する関数
def extract_numbers(text):
    # カンマと空白を削除し、数字のみを抽出
    numbers = re.findall(r'[\d.]+', text)
    
    # カンマを削除して数字を連結
    cleaned_number = ''.join(numbers).replace(',', '')
    
    return cleaned_number

# # 各要素から数字の部分を抽出
# result = [extract_numbers(item) for item in data]

# # 結果を表示
# for number in result:
#     print(number)


## まとめ

In [8]:
def extract_numbers(text):
    # カンマと空白を削除し、数字のみを抽出
    numbers = re.findall(r'[\d.]+', text)    
    # カンマを削除して数字を連結
    cleaned_number = ''.join(numbers).replace(',', '')
    
    return cleaned_number

def pbr_group(soup):
    tag = soup.find('div', id = "stockinfo_i3")
    tag_tds = tag.find_all('td')

    text_list = []
    for td in tag_tds:
        # print(td.get_text())
        text_list.append(extract_numbers(td.get_text()))
    
    return text_list[:-1]

def kairiritu(soup):
    tag = soup.find('div', class_="kabuka_trend clearfix")
    tr = tag.find_all('tr')[-1]
    tds = tr.find_all('td')

    text_list = []
    for td in tds:
    # print(td.get_text()[:-1])
        text_list.append(extract_numbers(td.get_text()[:-1]))
    
    return text_list

def volume_group(soup):
    tag = soup.find('div', id="kobetsu_left")
    tables = tag.find_all('table')

    text_list = []
    for table in tables[:2]:
        trs = table.find_all('tr')
        for tr in trs:
            # print(tr.find('td').get_text())
            text_list.append(extract_numbers(tr.find('td').get_text()))
    return [text_list[i] for i in [3, 4, 5, 6, 12, 13]]

def sector(soup):
    tag = soup.find('div', class_="company_block")
    trs = tag.find_all('tr')
    return [trs[3].td.get_text()]


In [66]:
codes = ['9505', '9508', '9511', '9509', '9507', '9503', '9504',
         '5020', '1605', '5013', '5401', '4208', '5851', '5703',
         '5541', '5711', '5406', '5017', '8306'] 

columns = ["code", "PER", "PBR", "Yield", "MarginRatio", "5days", "25days", "75days", "200days",
          "Close", "Volume", "TradeVolume", "VWAP", "MarginCap", "ShareOutstanding", "sector"]

In [8]:
response = requests.get(kabutan_kobetu + code)
soup = BeautifulSoup(response.content, "html.parser")

In [41]:
pbr_group(soup)

['9.6', '0.71', '', '10.29', '1931']

In [42]:
kairiritu(soup)

['5.86', '10.37', '14.32', '37.74']

In [47]:
volume_group(soup)

['918.3', '2214000', '2016', '910.787', '1931', '210333694']

In [9]:
for code in codes:
    response = requests.get(kabutan_kobetu + code)
    soup = BeautifulSoup(response.content, "html.parser")
    pbr_group(soup)
    kairiritu(soup)
    volume_group(soup)
    sector(soup)

In [10]:
dat = pd.DataFrame(columns=columns)
for code in codes:
    elements = [code]
    response = requests.get(kabutan_kobetu + code)
    soup = BeautifulSoup(response.content, "html.parser")
    elements.extend(pbr_group(soup))
    elements.extend(kairiritu(soup))
    elements.extend(volume_group(soup))
    elements.extend(sector(soup))
    
    tmp = pd.DataFrame([elements], columns=columns)
    dat = pd.concat([dat, tmp])

ValueError: 17 columns passed, passed data had 16 columns

In [70]:
dat

,code,PER,PBR,Yield,MarginRatio,5days,25days,75days,200days,Close,Volume,TradeVolume,VWAP,MarginCap,ShareOutstanding,sector
0,9505,9.6,0.71,,10.29,5.86,10.37,14.32,37.74,918.3,2214000,2016,910.787,1931,210333694,電気・ガス
0,9508,5.9,1.06,1.86,18.58,4.58,12.01,15.84,31.26,1077.0,4302600,4609,1071.328,5107,474183951,電気・ガス
0,9511,23.6,0.64,0.79,14.12,5.43,9.76,8.94,14.64,1258,619400,774,1249.211,716,56927965,電気・ガス
0,9509,5.8,0.76,,16.90,4.91,10.89,17.96,36.32,726.0,1985600,1434,722.257,1563,215291912,電気・ガス
0,9507,8.1,0.75,2.67,9.67,3.79,8.24,11.75,31.34,1121.5,1153000,1287,1116.544,2327,207528202,電気・ガス
0,9503,6.9,1.05,2.12,2.19,3.21,14.28,26.03,55.77,2355.0,8269800,19499,2357.845,22107,938733028,電気・ガス
0,9504,5.3,0.73,0.97,4.03,3.57,8.28,8.26,30.83,1032.5,3033600,3121,1028.804,3997,387154692,電気・ガス
0,5020,10.6,0.65,3.48,4.55,5.22,14.42,22.61,30.45,631.9,43269200,27428,633.882,19165,3032850649,石油・石炭
0,1605,9.3,0.70,3.21,2.97,2.52,10.12,26.59,45.07,2304.0,20257400,46886,2314.508,31949,1386667167,鉱業
0,5013,9.1,0.63,3.33,40.52,2.52,12.19,33.78,67.25,1654,167600,276,1647.367,230,13900065,石油・石炭


# 株情報スクレイピング完成

In [67]:
codes = ['9505', '9508', '9511', '9509', '9507', '9503', '9504',
         '5020', '1605', '5013', '5401', '4208', '5851', '5703',
         '5541', '5711', '5406', '5017', '8306'] 

columns = ["code", "Market", "PER", "PBR", "Yield", "MarginRatio", "5days", "25days", "75days", "200days",
          "Close", "Volume", "TradeVolume", "VWAP", "MarginCap", "ShareOutstanding", "sector"]

columns_light = ["code", "Market", "PER", "PBR", "Yield", "MarginRatio", "Close", "Volume", "TradeVolume", "VWAP", "MarginCap", "ShareOutstanding"]

In [32]:
def extract_numbers(text):
    # カンマと空白を削除し、数字のみを抽出
    numbers = re.findall(r'[\d.]+', text)    
    # カンマを削除して数字を連結
    cleaned_number = ''.join(numbers).replace(',', '')
    
    return cleaned_number

def pbr_group(soup):
    tag = soup.find('div', id = "stockinfo_i3")
    tag_tds = tag.find_all('td')

    text_list = []
    for td in tag_tds:
        # print(td.get_text())
        text_list.append(extract_numbers(td.get_text()))
    
    return text_list[:-1]

def kairiritu(soup):
    tag = soup.find('div', class_="kabuka_trend clearfix")
    tr = tag.find_all('tr')
    if tr == []:
        return ["", "", "", ""]
    else:
        tr = tag.find_all('tr')[-1]
        tds = tr.find_all('td')

    text_list = []
    for td in tds:
    # print(td.get_text()[:-1])
        text_list.append(extract_numbers(td.get_text()[:-1]))
    
    return text_list

def volume_group(soup):
    tag = soup.find('div', id="kobetsu_left")
    tables = tag.find_all('table')

    text_list = []
    for table in tables[:2]:
        trs = table.find_all('tr')
        for tr in trs:
            # print(tr.find('td').get_text())
            text_list.append(extract_numbers(tr.find('td').get_text()))
    return [text_list[i] for i in [3, 4, 5, 6, 12, 13]]

def sector(soup):
    tag = soup.find('div', class_="company_block")
    trs = tag.find_all('tr')
    return [trs[3].td.get_text()]

In [61]:
kabutan_kobetu = 'https://kabutan.jp/stock/?code='

def stock_info(code):
    response = requests.get(kabutan_kobetu + code)
    soup = BeautifulSoup(response.content, "html.parser")

    elements = [code]
    response = requests.get(kabutan_kobetu + code)
    soup = BeautifulSoup(response.content, "html.parser")
    elements.extend([soup.find('span', class_="market").get_text()])
    elements.extend(pbr_group(soup))
    elements.extend(kairiritu(soup))
    elements.extend(volume_group(soup))
    elements.extend(sector(soup))
    
    return pd.DataFrame([elements])

def stock_info_light(code):
    response = requests.get(kabutan_kobetu + code)
    soup = BeautifulSoup(response.content, "html.parser")

    elements = [code]
    response = requests.get(kabutan_kobetu + code)
    soup = BeautifulSoup(response.content, "html.parser")
    elements.extend([soup.find('span', class_="market").get_text()])
    elements.extend(pbr_group(soup))
    # elements.extend(kairiritu(soup))
    elements.extend(volume_group(soup))
    # elements.extend(sector(soup))
    
    return pd.DataFrame([elements])

In [84]:
stock_info("8366")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,8366,東証Ｐ,11.0,0.36,2.54,4.22,1.00,9.96,20.41,26.17,3550,126000,451,3577.496,1885,53090081,銀行業


## 業種別

## import nikkei code

In [4]:
import json

In [58]:
f = open('nikkei_code.json', 'r')
nikkei_dict = json.load(f)
f = open('nikkei_gyousyu_code.json', 'r')
nikkei_gyousyu_code = json.load(f)

## 株情報

In [12]:
kabutan_kobetu = 'https://kabutan.jp/stock/?code='

df = pd.DataFrame()
for i in range(9, 10):
    for code in nikkei_dict[str(i)]:
        dat = stock_info(code)
        df = pd.concat([df, dat])

In [13]:
df.columns = columns
numerical_columns = ['PER', 'PBR', 'Yield', 'MarginRatio',
                    'Volume', 'TradeVolume', 'MarginCap', 'ShareOutstanding']
# df[numerical_columns] = df[numerical_columns].astype(float)
df[numerical_columns] = df[numerical_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df

,code,Market,PER,PBR,Yield,MarginRatio,5days,25days,75days,200days,Close,Volume,TradeVolume,VWAP,MarginCap,ShareOutstanding,sector
0,3315,東証Ｐ,15.4,0.70,2.27,24.90,7.67,15.06,17.91,34.96,132,8454900,1123,132.823,399.0,302349449,石油・石炭
0,5013,東証Ｓ,9.1,0.63,3.33,40.52,2.52,12.19,33.78,67.25,1654,167600,276,1647.367,230.0,13900065,石油・石炭
0,5017,東証Ｐ,5.1,0.39,2.69,11.60,7.27,18.83,28.74,36.88,372,2287000,852,372.737,291.0,78183677,石油・石炭
0,5020,東証Ｐ,10.6,0.65,3.48,4.55,5.22,14.42,22.61,30.45,631.9,43269200,27428,633.882,19165.0,3032850649,石油・石炭
0,5011,東証Ｐ,11.0,0.85,3.36,4.15,1.88,0.82,5.79,26.12,2086,79900,166,2082.805,661.0,31685955,石油・石炭
0,5019,東証Ｐ,10.3,0.63,3.29,6.37,4.83,15.47,21.57,21.32,3646.0,3555600,13071,3676.172,1860.0,297864718,石油・石炭
0,5018,東証Ｐ,15.9,0.59,3.74,3.96,0.64,1.16,2.18,3.39,1203,16200,19,1200.944,117.0,9696500,石油・石炭
0,5015,東証Ｓ,25.3,2.08,3.98,473.00,0.80,2.46,2.87,1.38,905,17300,16,904.78,208.0,22975189,石油・石炭
0,5010,東証Ｓ,NaN,0.57,NaN,NaN,1.58,4.02,0.75,6.01,116,28400,3,115.208,26.0,22400000,石油・石炭
0,5021,東証Ｐ,9.1,0.97,4.34,18.31,1.07,8.60,27.52,38.44,5762,778200,4521,5809.932,5091.0,88353761,石油・石炭


In [18]:
# all_df = pd.DataFrame()
# for group, data in df.groupby('Market'):
#     # print(f'Category: {group}')
#     tmp = pd.concat([data, pd.DataFrame([data[numerical_columns].median()])], ignore_index=True)
#     all_df = pd.concat([all_df, tmp])

In [21]:
dat = df.groupby("Market")[numerical_columns].median()
dat

,PER,PBR,Yield,MarginRatio,Volume,TradeVolume,MarginCap,ShareOutstanding
Market,,,,,,,,
東証Ｐ,10.6,0.65,3.360,6.37,2287000.0,1123.0,661.0,88353761.0
東証Ｓ,17.2,0.63,3.655,256.76,28400.0,16.0,208.0,22400000.0


In [25]:
pd.concat([dat, dat], keys=["1", "2"])

PER   PBR  Yield  MarginRatio     Volume  TradeVolume  MarginCap  \
  Market                                                                      
1 東証Ｐ     10.6  0.65  3.360         6.37  2287000.0       1123.0      661.0   
  東証Ｓ     17.2  0.63  3.655       256.76    28400.0         16.0      208.0   
2 東証Ｐ     10.6  0.65  3.360         6.37  2287000.0       1123.0      661.0   
  東証Ｓ     17.2  0.63  3.655       256.76    28400.0         16.0      208.0   

          ShareOutstanding  
  Market                    
1 東証Ｐ           88353761.0  
  東証Ｓ           22400000.0  
2 東証Ｐ           88353761.0  
  東証Ｓ           22400000.0

In [81]:
df_list = []
for i in nikkei_gyousyu_code.keys():
    df = pd.DataFrame()
    for code in nikkei_dict[i]:
        dat = stock_info_light(code)
        df = pd.concat([df, dat])
    df.columns = columns_light
    df[numerical_columns] = df[numerical_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
    df_list.append(df.groupby("Market")[numerical_columns].median())

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [88]:
df_list[26]

,PER,PBR,Yield,MarginRatio,Volume,TradeVolume,MarginCap,ShareOutstanding
Market,,,,,,,,
名証Ｍ,255.90,3.49,NaN,NaN,4900.0,8.0,105.60,14204109.0
名証Ｎ,49.50,2.33,2.340,NaN,2500.0,2.0,14.90,6451000.0
札証Ａ,7.20,5.21,NaN,NaN,93900.0,34.0,507.35,278768574.0
東証Ｇ,22.45,2.84,1.105,10.25,20350.0,23.0,58.20,7263500.0
東証Ｐ,19.15,1.59,1.660,2.32,144650.0,232.0,741.50,40007261.0
東証Ｓ,20.10,1.33,1.825,5.43,18900.0,13.0,68.90,10458000.0
福証,10.20,4.30,0.980,NaN,10500.0,11.0,325.00,31931900.0


In [90]:
df_list_new = []
for i in range(27, 34):
    df = pd.DataFrame()
    for code in nikkei_dict[str(i)]:
        dat = stock_info_light(code)
        df = pd.concat([df, dat])
    df.columns = columns_light
    df[numerical_columns] = df[numerical_columns].apply(lambda x: pd.to_numeric(x, errors='coerce'))
    df_list_new.append(df.groupby("Market")[numerical_columns].median())

In [93]:
df_list[1]

,PER,PBR,Yield,MarginRatio,Volume,TradeVolume,MarginCap,ShareOutstanding
Market,,,,,,,,
東証Ｐ,9.3,0.70,2.78,4.66,370500.0,1069.0,907.0,28336061.0
東証Ｓ,6.5,1.31,1.66,1151.00,1217400.0,543.0,267.0,58892853.0


In [100]:
a = [2,3]
b = [4, 3]

In [101]:
a.extend(b)
a

[2, 3, 4, 3]

In [ ]:
test = 